In [1]:
pip install qiskit==0.46.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 23.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 10.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install qiskit-aer==0.14.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install qiskit-terra==0.46.1


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scipy==1.11.4

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
from qiskit import Aer, execute
from scipy.optimize import minimize
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import Statevector
from qiskit.quantum_info import SparsePauliOp
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.algorithms import VQE
from qiskit.quantum_info import random_clifford
from qiskit import QuantumCircuit

seed = 1
np.random.seed(seed)
operator_matrix = np.matrix([[4.12622481 + 0.j, -0.08968504 + 4.79784639j, 1.65775982 + 0.92708254j,
                              1.17624349 + 2.4661674j, -3.335948 + 1.08703469j, -4.98109937 + 0.70532393j,
                              2.71579923 + 4.58174919j, 1.0245947 + 4.28887226j],
                             [-0.08968504 - 4.79784639j, -1.52015431 + 0.j, 3.53812825 + 1.04090814j,
                              -0.62529457 - 0.28077156j, -3.11655239 + 4.72991548j, 0.85201655 - 4.6724328j,
                              -1.96061315 + 1.49140336j, 4.81993641 - 0.02042725j],
                             [1.65775982 - 0.92708254j, 3.53812825 - 1.04090814j, 4.73317403 + 0.j,
                              2.69118181 + 0.69165245j, 2.17508288 + 0.37789998j, 3.75557884 + 1.37651403j,
                              -4.50761077 - 2.24874379j, 2.20380462 + 1.60226692j],
                             [1.17624349 - 2.4661674j, -0.62529457 + 0.28077156j, 2.69118181 - 0.69165245j,
                              0.2992812 + 0.j, -4.12634487 + 0.45647952j, -2.925992 + 4.25854082j,
                              0.97069172 - 2.03058852j, 0.04904204 + 0.03472227j],
                             [-3.335948 - 1.08703469j, -3.11655239 - 4.72991548j, 2.17508288 - 0.37789998j,
                              -4.12634487 - 0.45647952j, 2.09632259 + 0.j, 4.90225303 - 2.18312633j,
                              -1.0118842 - 3.61389139j, -0.3231792 - 1.28326535j],
                             [-4.98109937 - 0.70532393j, 0.85201655 + 4.6724328j, 3.75557884 - 1.37651403j,
                              -2.925992 - 4.25854082j, 4.90225303 + 2.18312633j, -4.4317145 + 0.j,
                              -3.97538473 - 2.09544471j, -4.18585618 - 4.33025063j],
                             [2.71579923 - 4.58174919j, -1.96061315 - 1.49140336j, -4.50761077 + 2.24874379j,
                              0.97069172 + 2.03058852j, -1.0118842 + 3.61389139j, -3.97538473 + 2.09544471j,
                              1.95815546 + 0.j, 1.93252228 - 4.61344071j],
                             [1.0245947 - 4.28887226j, 4.81993641 + 0.02042725j, 2.20380462 - 1.60226692j,
                              0.04904204 - 0.03472227j, -0.3231792 + 1.28326535j, -4.18585618 + 4.33025063j,
                              1.93252228 + 4.61344071j, -1.19669648 + 0.j]])
operator_matrix = np.matrix([[0, 1, 0, -1.j], [1, 0, 1.j, 0], [0, -1.j, 0, -1], [1.j, 0, -1, 0]])
operator_matrix = np.matrix([[3. + 0.j, 2. + 0.j, 0. + 0.j, 0. - 2.5j], [2. + 0.j, -3. + 0.j, 0. + 2.5j, 0. + 0.j],
                             [0. + 0.j, 0. - 2.5j, -3. + 0.j, 0. + 0.j], [0. + 2.5j, 0. + 0.j, 0. + 0.j, 3. + 0.j]])
pauli_operator = SparsePauliOp.from_operator(operator_matrix)


def prepare_meas_circuits(circuit: QuantumCircuit, n_shadows, seed=None):
    """Append n_shadows random clifford unitaries to the circuit
    returns collection of circuits along with collection of the unitaries used"""

    circuits = []
    unitaries = []
    for i in range(n_shadows):
        clifford = random_clifford(num_qubits=circuit.num_qubits, seed=seed)
        cir = circuit.compose(clifford.to_circuit())
        cir.measure_all()
        circuits.append(cir)
        unitaries.append(clifford.to_matrix())

    return circuits, unitaries


def estimate_fidelity_from_shadow(unitaries, measured_bitstrings, target_state):
    """Fidelity estimation with classical shadows

    Implemented as per Approximate complex amplitude encoding algorithm and its application to data classification problems.
    Mitsuda et al. https://arxiv.org/abs/2211.13039 (page 2, equation (10))"""

    measured_bitstrings = [np.matrix(b) for b in measured_bitstrings]
    unitaries = [np.matrix(u) for u in unitaries]
    target_state = np.matrix(target_state)

    n_shots = len(measured_bitstrings)
    n_qubits = np.log2(target_state.shape[1])
    observable = target_state.T @ np.conj(target_state)

    fidelity = 0
    factor = 2 ** n_qubits + 1
    for i in range(n_shots):
        product = np.conj(measured_bitstrings[i]) @ (unitaries[i] @ observable @ unitaries[i].H) @ measured_bitstrings[
            i].T
        cur_fid = factor * product - 1
        # print(f"{cur_fid} -- {measured_bitstrings[i]}")
        fidelity += cur_fid

    fidelity = fidelity / n_shots  # /(2**n_qubits)
    fidelity = np.abs(np.sum(fidelity))
    return fidelity





def generate_hermitian(n=2, a_range=(-5, 5), complex=True, sparsity=None):
    shape = (2 ** n, 2 ** n)

    # generate random matrix
    matrix = generate_matrix(shape, a_range, complex, sparsity)

    # make it hermitian
    for i in range(shape[0]):
        for j in range(shape[1]):
            if i == j:
                # values on diagonal must be real
                matrix[i, j] = np.real(matrix[i, j])
            if i > j:
                # values below diagonal must be conjugate of corresponding values above diagonal
                matrix[i, j] = np.conjugate(matrix[j, i])

    return matrix


def generate_matrix(shape=(2, 2), a_range=(-5, 5), complex=True, sparsity=None):
    # generate uniform random (complex) matrix

    matrix = np.random.uniform(*a_range, shape)
    if complex:
        matrix = matrix + 1.j * np.random.uniform(*a_range, shape)
    # apply sparsity
    if sparsity:
        for i in range(shape[0]):
            for j in range(shape[1]):
                matrix[i, j] = np.random.choice([0, matrix[i, j]], p=[sparsity, 1 - sparsity])

    return np.matrix(matrix)


def power_method(matrix, num_iterations=10):  # by power iteration
    # choose random initial vector
    bk = np.random.rand(matrix.shape[1]) + 1.j * np.random.rand(matrix.shape[1])
    bk = np.matrix(bk).T

    for i in range(num_iterations):
        bk1 = np.dot(matrix, bk)
        bk = bk1 / np.linalg.norm(bk1)

    return bk


def inverse_power_method(matrix, shift=0, num_iterations=10):
    inv_matrix = np.linalg.inv(matrix - shift * np.identity(matrix.shape[0]))
    return power_method(inv_matrix, num_iterations)


def get_gershgorins(matrix, draw=False):
    centers, radii = [], []
    for i in range(matrix.shape[0]):
        centers.append(matrix[i, i])
        radii.append(np.sum([np.abs(matrix[i, j]) if j is not i else 0 for j in range(matrix.shape[0])]))
    return centers, radii


def get_gershgorin_extrema(matrix):
    centers, radii = get_gershgorins(matrix, draw=False)
    min_bound = min(np.subtract(np.real(centers), radii))
    max_bound = max(np.add(np.real(centers), radii))
    return min_bound, max_bound



def main():
    # 1. classical: approximate solution
    approx_iterations = 3  # for inverse power method to obtain approx eigenvector
    shift = get_gershgorin_extrema(operator_matrix)[0]
    approx_eigenstate = inverse_power_method(operator_matrix, shift=shift, num_iterations=approx_iterations).T.tolist()[
        0]

    # 2. VQA: pre-train ansatz with ACAE
    # 2.1. circuit generation
    n_shadows = 100
    problem_size = int(np.log2(len(operator_matrix)))
    ansatz = EfficientSU2(num_qubits=problem_size, reps=2)
    meas_circuits, unitaries = prepare_meas_circuits(ansatz, n_shadows, seed=seed)

    # 2.2. optimize (ACAE)
    initial_param_values = np.random.uniform(-np.pi, np.pi, size=len(ansatz.parameters))
    print(initial_param_values)
    backend = Aer.get_backend("statevector_simulator")

    def objective(x):
        final_meas_circuits = [c.assign_parameters(x) for c in meas_circuits]
        job = execute(final_meas_circuits, backend=backend, shots=1, seed_transpiler=seed, seed_simulator=seed)
        res = job.result()
        counts = [res.get_counts(i) for i, c in enumerate(final_meas_circuits)]
        bs = [Statevector.from_label(list(cnt.keys())[0]) for cnt in counts]
        obj_value = -estimate_fidelity_from_shadow(unitaries, bs, approx_eigenstate)
        # print(f"current params = {list(x)}")
        print("ACAE fidelity", obj_value)
        return obj_value

    # start messung
    res = minimize(objective, initial_param_values, method="COBYLA",
                   options=({"rhobeg": np.pi / 4, "maxiter": 20, "tol": np.pi / (2 ** 6)}))
    # ende messung
    print(res)

    # Needed?
    # 3. VQA: VQE with parameter initialization from ACAE
    # 3.1. circuit generation
    vqe = VQE(ansatz=ansatz)  # reuse same ansatz
    vqe_circuits = vqe.construct_circuit(ansatz.parameters, PauliSumOp(pauli_operator))
    pauli_lists = pauli_operator.paulis.group_commuting(
        qubit_wise=True)  # contains the respective pauli operators that can be estimated from the measument outcomes of each circuit
    print(vqe_circuits)

    # 3.2. optimize (VQE)
    initial_param_values = res.x  # use parameterization obtained from pre-training

    def objective(x):
        final_meas_circuits = [c.assign_parameters(x) for c in vqe_circuits]
        job = execute(final_meas_circuits, backend=backend, seed_transpiler=seed, seed_simulator=seed)
        res = job.result()

        exp = 0.0
        for i in range(len(final_meas_circuits)):
            paulis = pauli_lists[i]
            counts = res.get_counts(i)

            # compute contribution of each pauli operator (p_exp)
            for pauli in paulis:
                p_exp = 0.0
                sum_shots = sum([v for _, v in counts.items()])  # find out how many shots were executed in total
                for k, v in counts.items():
                    k_pruned = [b for j, b in enumerate(k) if str(pauli)[j] != 'I']  # remove bits where operator is I
                    p_exp += (-1) ** (k_pruned.count('1')) * v / sum_shots

                # add to overall expectation weighted by coefficient for respective pauli operator
                idx = pauli_operator.paulis.settings["data"].index(str(pauli))
                exp += pauli_operator.coeffs[idx] * p_exp

        print("VQE expectation", exp.real)
        return exp.real

    res = minimize(objective, initial_param_values, method="COBYLA", options={"rhobeg": 3 * np.pi / 16, "maxiter": 50})
    print(res)


main()

### Qiskit's VQE implementation for reference
# from qiskit_aer.primitives import Estimator as AerEstimator
# from qiskit_algorithms.minimum_eigensolvers import VQE
# from qiskit_algorithms.optimizers import COBYLA
# noiseless_estimator = AerEstimator(run_options={"seed": seed, "shots": None},)
# optimizer = COBYLA(maxiter=50, rhobeg=3*np.pi/16)
# vqe = VQE(noiseless_estimator, ansatz, optimizer=optimizer, initial_point=initial_param_values)
# result = vqe.compute_minimum_eigenvalue(operator=pauli_operator)
# print(result)
# objective(result.optimal_parameters)







/var/folders/sx/5fkfsh9s4xsg76n37nm652pm0000gn/T/ipykernel_20408/23137701.py:7: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed in Qiskit 1.0. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow.primitive_ops import PauliSumOp
/var/folders/sx/5fkfsh9s4xsg76n37nm652pm0000gn/T/ipykernel_20408/23137701.py:8: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed in Qiskit 1.0. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms import VQE


[-0.64862909  0.24389273 -0.50771584  1.16376844 -1.85698128  2.37578192
 -2.96951133  1.07107896 -0.51958925  0.36875907 -2.2595155  -1.89688429]


/var/folders/sx/5fkfsh9s4xsg76n37nm652pm0000gn/T/ipykernel_20408/23137701.py:175: DeprecationWarning: The 'qiskit.Aer' entry point is deprecated and will be removed in Qiskit 1.0. You should use 'qiskit_aer.Aer' directly instead.
  backend = Aer.get_backend("statevector_simulator")
/var/folders/sx/5fkfsh9s4xsg76n37nm652pm0000gn/T/ipykernel_20408/23137701.py:179: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(final_meas_circuits, backend=backend, shots=1, seed_transpiler=seed, seed_simulator=seed)


ACAE fidelity -0.292962166415385


/var/folders/sx/5fkfsh9s4xsg76n37nm652pm0000gn/T/ipykernel_20408/23137701.py:179: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(final_meas_circuits, backend=backend, shots=1, seed_transpiler=seed, seed_simulator=seed)


...

VQE expectation -5.722815521879006
VQE expectation -5.6907218402256925
VQE expectation -5.74992651099223


/var/folders/sx/5fkfsh9s4xsg76n37nm652pm0000gn/T/ipykernel_20408/23137701.py:207: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(final_meas_circuits, backend=backend, seed_transpiler=seed, seed_simulator=seed)
/var/folders/sx/5fkfsh9s4xsg76n37nm652pm0000gn/T/ipykernel_20408/23137701.py:207: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(final_meas_circuits, 

VQE expectation -5.69634737531305
VQE expectation -5.7429059876475534
VQE expectation -5.7180645436991995


/var/folders/sx/5fkfsh9s4xsg76n37nm652pm0000gn/T/ipykernel_20408/23137701.py:207: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(final_meas_circuits, backend=backend, seed_transpiler=seed, seed_simulator=seed)
/var/folders/sx/5fkfsh9s4xsg76n37nm652pm0000gn/T/ipykernel_20408/23137701.py:207: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(final_meas_circuits, 

VQE expectation -5.743687909731607
 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -5.74992651099223
       x: [ 1.853e-01  8.812e-01 -1.276e-01  5.706e-01 -1.864e+00
            2.799e+00 -3.541e+00  2.433e+00 -1.042e+00  7.443e-01
           -2.502e+00 -1.738e+00]
    nfev: 50
   maxcv: 0.0
